In [1]:
import numpy as np
import pandas as pd
import evaluate
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss

from preprocessing import climateBUG_reduce_rows



In [2]:
# This should always output true now, but worth checking
print(f"is GPU available: {torch.cuda.is_available()}")

is GPU available: True


In [3]:
# The climate-fever dataset is in a format which the transformers Trainer does not understand
# It must be preprocessed using the functions in preprocessing.py

train_df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-training-dataset.json")
preprocessed_training_df = climateBUG_reduce_rows(train_df, rows=10000)

test_df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-testing-dataset.json")
preprocessed_testing_df = climateBUG_reduce_rows(test_df, rows=1000)


                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Assurances   captured  fur...      0
3     Our  business  is  also  exposed  to  the  ups...      0
4     Crédit  Agricole  assumed  its   responsibilit...      0
...                                                 ...    ...
9995  Other general administrative expenses (Note 33...      0
9996  Other financial instruments not measured at fa...      0
9997             Revaluation gains/(losses) 191 (1,054)      0
9998  Final balance at 31.12.2014 5,714,956 12,032,8...      0
9999                Tangible assets (218,340) (249,701)      0

[10000 rows x 2 columns]
                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Ass

In [4]:
preprocessed_training_df.loc[0]

statement    2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...
label                                                        0
Name: 0, dtype: object

In [5]:
preprocessed_testing_df.loc[0]

statement    4CRÉDIT AGRICOLE S.A. – 2020-2021 INTEGRATED R...
label                                                        0
Name: 0, dtype: object

In [6]:
training_dataset = Dataset.from_pandas(preprocessed_training_df)
testing_dataset = Dataset.from_pandas(preprocessed_testing_df)
training_dataset[0]

{'statement': '2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REPORT2 CRÉDIT AGRICOLE S.A.      2015 REGISTRATION DOCUMENT MESSAGE FROM THE CHAIRMAN AND THE CHIEF EXECUTIVE OFFICER MESSAGE FROM THE CHAIRMANAND THE CHIEF EXECUTIVE OFFICERPHILIPPE BRASSAC, CHIEF EXECUTIVE OFFICER AND DOMINIQUE LEFEBVRE, CHAIRMAN OF CRÉDIT AGRICOLE S.A.A robust Group delivering sustainable performanceLeading  a  bank  with  a  EUR  698  billion   loan   book(1)',
 'label': 0}

In [7]:
training_dataset = training_dataset.shuffle(seed=12)
testing_dataset = testing_dataset.shuffle(seed=12)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
def custom_tokenize(examples):
    tokenized_output = tokenizer(
        text=examples["statement"],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_training_dataset = training_dataset.map(custom_tokenize, batched=True)
tokenized_testing_dataset = testing_dataset.map(custom_tokenize, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
tokenized_training_dataset

Dataset({
    features: ['statement', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [11]:

print(tokenized_training_dataset[0]["statement"])
print(tokenized_training_dataset[0]["label"])
print(tokenized_training_dataset[0]["input_ids"])
print(tokenized_training_dataset[0]["attention_mask"])

print(tokenized_testing_dataset[9]["statement"])
print(tokenized_testing_dataset[9]["label"])
print(tokenized_testing_dataset[9]["input_ids"])
print(tokenized_testing_dataset[9]["attention_mask"])

R. 225-105-1 I 3° a)465 – 469; 490 – 492TravelG4-EN30148-3 1°487 – 488Environmental approachArt.
0
[0, 500, 4, 50141, 18614, 12, 15274, 12, 134, 38, 155, 12938, 10, 43, 33544, 1437, 50300, 204, 4563, 131, 40569, 1437, 50300, 2766, 176, 39258, 534, 306, 12, 2796, 28167, 3818, 12, 246, 50141, 134, 12938, 37795, 1437, 50300, 204, 4652, 45311, 1437, 50290, 23295, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "climatebert/distilroberta-base-climate-f",
    num_labels=2,
    # ignore_mismatched_sizes=True
).to("cuda")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
print(next(model.parameters()).device)

cuda:0


In [14]:
model.gradient_checkpointing_enable()

In [15]:
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [16]:
# This is where we set the hyperparameters
training_args = TrainingArguments(
    output_dir="./results/climateBERT-base/climateBUG/seed12",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="no",
    save_steps=500,
    fp16=True,                          # Use 16-bit floating point instead of 32 - makes computation faster
    warmup_ratio=0.05,                    # Allows the model to adapt a little
    # gradient_accumulation_steps=2       # Might help with OOM errors, if we have them
    learning_rate=3e-5,
    push_to_hub=False,
)

In [17]:
trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [18]:
trainer.train()

  0%|          | 0/3750 [00:00<?, ?it/s]

{'loss': 0.6218, 'grad_norm': 3.6418814659118652, 'learning_rate': 7.978723404255319e-06, 'epoch': 0.04}
{'loss': 0.5401, 'grad_norm': 5.092080116271973, 'learning_rate': 1.5957446808510637e-05, 'epoch': 0.08}
{'loss': 0.3433, 'grad_norm': 31.030418395996094, 'learning_rate': 2.3617021276595744e-05, 'epoch': 0.12}
{'loss': 0.3149, 'grad_norm': 10.121904373168945, 'learning_rate': 2.9915777653003933e-05, 'epoch': 0.16}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.32883888483047485, 'eval_accuracy': 0.858, 'eval_f1_score': 0.8495138967383093, 'eval_runtime': 9.3647, 'eval_samples_per_second': 106.784, 'eval_steps_per_second': 13.348, 'epoch': 0.16}
{'loss': 0.2805, 'grad_norm': 6.079509258270264, 'learning_rate': 2.950308815272319e-05, 'epoch': 0.2}
{'loss': 0.3236, 'grad_norm': 11.648209571838379, 'learning_rate': 2.908197641774284e-05, 'epoch': 0.24}
{'loss': 0.249, 'grad_norm': 5.566133499145508, 'learning_rate': 2.8660864682762495e-05, 'epoch': 0.28}
{'loss': 0.2258, 'grad_norm': 8.68155574798584, 'learning_rate': 2.8239752947782145e-05, 'epoch': 0.32}


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.4521620571613312, 'eval_accuracy': 0.906, 'eval_f1_score': 0.9070983554961051, 'eval_runtime': 9.5748, 'eval_samples_per_second': 104.441, 'eval_steps_per_second': 13.055, 'epoch': 2.72}
{'loss': 0.1364, 'grad_norm': 0.08435054123401642, 'learning_rate': 2.5687815833801235e-06, 'epoch': 2.76}
{'loss': 0.1711, 'grad_norm': 1.0449551343917847, 'learning_rate': 2.1476698483997755e-06, 'epoch': 2.8}
{'loss': 0.12, 'grad_norm': 31.33572006225586, 'learning_rate': 1.7265581134194273e-06, 'epoch': 2.84}
{'loss': 0.1196, 'grad_norm': 10.738176345825195, 'learning_rate': 1.3054463784390791e-06, 'epoch': 2.88}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.4374213218688965, 'eval_accuracy': 0.906, 'eval_f1_score': 0.9069108635097494, 'eval_runtime': 9.5573, 'eval_samples_per_second': 104.632, 'eval_steps_per_second': 13.079, 'epoch': 2.88}
{'loss': 0.1269, 'grad_norm': 19.89076805114746, 'learning_rate': 8.843346434587311e-07, 'epoch': 2.92}
{'loss': 0.1379, 'grad_norm': 0.1526726931333542, 'learning_rate': 4.6322290847838293e-07, 'epoch': 2.96}
{'loss': 0.154, 'grad_norm': 2.3515231609344482, 'learning_rate': 4.211117349803481e-08, 'epoch': 3.0}
{'train_runtime': 1391.7666, 'train_samples_per_second': 21.555, 'train_steps_per_second': 2.694, 'train_loss': 0.2095044241587321, 'epoch': 3.0}


TrainOutput(global_step=3750, training_loss=0.2095044241587321, metrics={'train_runtime': 1391.7666, 'train_samples_per_second': 21.555, 'train_steps_per_second': 2.694, 'total_flos': 3974021959680000.0, 'train_loss': 0.2095044241587321, 'epoch': 3.0})

In [19]:
# trainer.save_model("./results/climateBERT-base/climateBUG/first_run")
# Please remember to delete model.safetensors BEFORE adding to git. Causes issues...
# Also it is probably not worth running this block until the model is worth keeping

In [20]:
# Metrics are not included in the save model so we need to save them separately
metrics = trainer.evaluate()
with open("./results/climateBERT-base/climateBUG/seed12/eval_metrics.json", "w") as output_file:
    json.dump(metrics, output_file)

  0%|          | 0/125 [00:00<?, ?it/s]